In [10]:
import json

with open("domvqa_verified.json", "r", encoding="utf-8") as f1:
    data1 = json.load(f1)
with open("domvqa_verified_v2.json", "r", encoding="utf-8") as f2:
    data2 = json.load(f2)

fused = data1 + data2

print(f"Loaded {len(data1)} from domvqa_verified.json")
print(f"Loaded {len(data2)} from domvqa_verified_v2.json")
print(f"Fused total: {len(fused)} examples")

# Remove samples where either dom_difficulty or img_difficulty is 0
filtered = [
    ex for ex in fused
    if ex.get("dom_difficulty", 1) != 0 and ex.get("img_difficulty", 1) != 0
]
print(f"Filtered total: {len(filtered)} examples (removed {len(fused) - len(filtered)})")

with open("domvqa_fused_filtered.json", "w", encoding="utf-8") as f:
    json.dump(filtered, f, ensure_ascii=False, indent=2)
print(f"Saved {len(filtered)} filtered examples to domvqa_fused_filtered.json")

Loaded 201 from domvqa_verified.json
Loaded 401 from domvqa_verified_v2.json
Fused total: 602 examples
Filtered total: 452 examples (removed 150)
Saved 452 filtered examples to domvqa_fused_filtered.json


In [11]:
import json

with open("domvqa_fused_filtered.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [12]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [13]:
from pydantic import BaseModel

class shortenedContext(BaseModel):
    shortened_context: str

In [14]:
def get_content(dom, question):
    return [
                { "type": "input_text", "text": f"""Given the dom <dom>{dom}</dom> 
                 and the question <question>{question}</question>, 
                 extract a shortened DOM content of around 100 lines. That content should be enough to answer the question.
                 It should of course include more than just the answer to the question, to still make it challenging and useful as a learning resource.
                 Copy from the DOM the content, and do not include any other text within the shortened content to output.
                 Now output.""" },
            ]

In [17]:
import json
import os

results = []

for i in range(202, len(data)):
    dom = data[i]["cleaned_html"]
    question = data[i]["question"]

    response = client.responses.parse(
        model="o3",
        input=[
            {
                "role": "user",
                "content": get_content(dom, question)
            }
        ],
        text_format=shortenedContext,
    )

    output_text = response.output_parsed.shortened_context

    print(output_text)

    result_dict = data[i]
    result_dict["shortened_context"] = output_text


    results.append(result_dict)
    if i % 50 == 0:
        print(f"Saved {i} results")
        with open("domvqa_verified_full_v2.json", "w") as f:
            json.dump(results, f, indent=2)

with open("domvqa_verified_full_v2.json", "w") as f:
    json.dump(results, f, indent=2)

<div backend_node_id="85650">
  <h1 backend_node_id="85653">
    <text backend_node_id="85654">Marketplace</text>
  </h1>
  <nav backend_node_id="85656" aria_label="Manage">
    <ul backend_node_id="85659" role="list">
      <li backend_node_id="85661">
        <a backend_node_id="85663">
          <span backend_node_id="85665">
            <text backend_node_id="85666">All Items</text>
          </span>
        </a>
      </li>
      <li backend_node_id="85670">
        <a backend_node_id="85672">
          <span backend_node_id="85674">
            <text backend_node_id="85675">Items I Want</text>
          </span>
        </a>
      </li>
      <li backend_node_id="85679">
        <a backend_node_id="85681">
          <span backend_node_id="85683">
            <text backend_node_id="85684">Purchases</text>
          </span>
        </a>
      </li>
      <li backend_node_id="85688">
        <a backend_node_id="85690" role="heading">
          <span backend_node_id="85692">
         

ValidationError: 1 validation error for shortenedContext
  Invalid JSON: EOF while parsing a string at line 1 column 2683 [type=json_invalid, input_value='{"shortened_context":"<s...          <text backend', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid

In [18]:
with open("domvqa_verified_full_v2.json", "r") as f:
    loaded_results = json.load(f)
print(f"Length of domvqa_verified_full.json: {len(loaded_results)}")


Length of domvqa_verified_full.json: 199
